# **Bitcoin price prediction - Block Split**
### Big Data Computing final project - A.Y. 2022 - 2023
Prof. Gabriele Tolomei

MSc in Computer Science

La Sapienza, University of Rome

### Author: Corsi Danilo (1742375) - corsi.1742375@studenti.uniroma1.it


---


Description: perform model's train / validation with hyperparameter tuning and cross validation based on different methods of splitting the dataset.

# Global constants, dependencies, libraries and tools

In [1]:
# Main constants
LOCAL_RUNNING = True
SLOW_OPERATIONS = False # Decide whether or not to use operations that might slow down notebook execution
ROOT_DIR = "D:/Documents/Repository/BDC/project" if LOCAL_RUNNING else "/content/drive"

In [2]:
if not LOCAL_RUNNING:
    # Point Colaboratory to Google Drive
    from google.colab import drive

    # Define GDrive paths
    drive.mount(ROOT_DIR, force_remount=True)

    # Install Spark and related dependencies
    !pip install pyspark
    !pip install -U -q PyDrive -qq
    !apt install openjdk-8-jdk-headless -qq

## Import my utilities

In [3]:
# Set main dir
MAIN_DIR = ROOT_DIR + "" if LOCAL_RUNNING else ROOT_DIR + "/MyDrive/BDC/project"

# Utilities dir
UTILITIES_DIR = MAIN_DIR + "/utilities"

# Import my utilities
import sys
sys.path.append(UTILITIES_DIR)

from imports import *
from config import *
import train_validation_utilities

importlib.reload(train_validation_utilities)

<module 'train_validation_utilities' from 'D:\\Documents/Repository/BDC/project/utilities\\train_validation_utilities.py'>

## Core variables

In [4]:
# BS = Block Split
# WFS = Walk Forward Split
# SS = Single Split
SPLITTING_METHOD = BS

# LR = LinearRegression 
# GLR = GeneralizedLinearRegression 
# RF = RandomForestRegressor 
# GBTR = GradientBoostingTreeRegressor
MODEL_NAME = GBTR

In [5]:
###################
# --- DATASET --- #
###################

# Datasets dirs
DATASET_OUTPUT_DIR = MAIN_DIR + "/datasets/output"

# Datasets paths
DATASET_TRAIN_VALID  = DATASET_OUTPUT_DIR + "/" + DATASET_TRAIN_VALID_NAME + ".parquet"

####################
# --- FEATURES --- #
####################

# Features dir
FEATURES_DIR = MAIN_DIR + "/features"

# Features paths
FEATURES_CORRELATION = FEATURES_DIR + "/" + FEATURES_CORRELATION_LABEL + ".json"
BASE_FEATURES = FEATURES_DIR + "/" + BASE_FEATURES_LABEL + ".json"
BASE_AND_MOST_CORR_FEATURES = FEATURES_DIR + "/" + BASE_AND_MOST_CORR_FEATURES_LABEL + ".json"
BASE_AND_LEAST_CORR_FEATURES = FEATURES_DIR + "/" + BASE_AND_LEAST_CORR_FEATURES_LABEL + ".json"

##################
# --- MODELS --- #
##################

# Model dir
MODELS_DIR = MAIN_DIR + "/models"

# Model path
MODEL = MODELS_DIR + "/" + MODEL_NAME

###################
# --- RESULTS --- #
###################

# Results dir
RESULTS_DIR = MAIN_DIR + "/results/" + SPLITTING_METHOD

# Results path
ALL_MODEL_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + "_all.csv"
REL_MODEL_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + "_rel.csv"

MODEL_ACCURACY_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + "_accuracy.csv"

In [6]:
# Importing useful libraries
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pio.renderers.default = 'vscode+colab' # To correctly render plotly plots

# Create the pyspark session

In [7]:
# Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '12G').\
                set('spark.driver.memory', '12G').\
                set('spark.driver.maxResultSize', '109G').\
                set("spark.kryoserializer.buffer.max", "1G").\
                setAppName("BitcoinPricePrediction").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

# Loading dataset

In [8]:
# Load train / validation set into pyspark dataset objects
df = spark.read.load(DATASET_TRAIN_VALID,
                         format="parquet",
                         sep=",",
                         inferSchema="true",
                         header="true"
                    )

In [9]:
train_validation_utilities.dataset_info(df)

+-------------------+---+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+-----------------+--------------------+--------------------+------------------+------------------+--------------------+------------------------+-------------------+------------------+--------------------+--------------------+------------------+--------------+--------------------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|          timestamp| id|     market-price|    opening-price|    highest-price|     lowest-price|    closing-price|  trade-volume-btc|   total-bitcoins|          market-cap|    trade-volume-usd|       blocks-size|    avg-block-size|n-transactions-total|n-transactions-per-block|          hash-rate|        difficulty|      miners-revenue|transaction-fees-usd|n-unique-addresses|n-transactions|estimated-transaction-volume-usd|       sma-5-days|   

# Loading features

In [10]:
# Loading base features
with open(BASE_FEATURES, "r") as f:
    BASE_FEATURES = json.load(f)
print(BASE_FEATURES)

['opening-price', 'highest-price', 'lowest-price', 'closing-price', 'trade-volume-btc', 'market-price', 'market-cap', 'total-bitcoins', 'trade-volume-usd']


In [11]:
# Loading currency and additional most correlated features
with open(BASE_AND_MOST_CORR_FEATURES, "r") as f:
    BASE_AND_MOST_CORR_FEATURES = json.load(f)
print(BASE_AND_MOST_CORR_FEATURES)

['opening-price', 'highest-price', 'lowest-price', 'closing-price', 'trade-volume-btc', 'market-price', 'market-cap', 'total-bitcoins', 'trade-volume-usd', 'miners-revenue', 'sma-5-days', 'sma-7-days', 'sma-10-days', 'estimated-transaction-volume-usd', 'sma-20-days']


In [12]:
# Loading currency and additional least correlated features
with open(BASE_AND_LEAST_CORR_FEATURES, "r") as f:
    BASE_AND_LEAST_CORR_FEATURES = json.load(f)
print(BASE_AND_LEAST_CORR_FEATURES)

['opening-price', 'highest-price', 'lowest-price', 'closing-price', 'trade-volume-btc', 'market-price', 'market-cap', 'total-bitcoins', 'trade-volume-usd', 'sma-100-days', 'transaction-fees-usd', 'n-unique-addresses', 'sma-50-days', 'n-transactions-total', 'blocks-size', 'hash-rate', 'difficulty', 'avg-block-size', 'n-transactions-per-block', 'n-transactions']


# Model train / validation
In order to train and validate the model I'll try several approaches:
- `Default without normalization:` make predictions using the base model
- `Default with normalization:` like the previous one but features are normalized

Then the features that gave on average the most satisfactory results (for each model) are chosen and proceeded with:
- `Hyperparameter tuning:` finding the best parameters to use. 
- `Cross Validation:` validate the performance of the model with the chosen parameters (also here using Block split / Walk forward split)

If the final results are satisfactory, the model will be trained on the whole train / validation set and saved in order to make predictions on the test set.

For each approach the train / validation set will be split according to the chosen splitting method (in order to figure out which one works best for our problem). In this case the `Block time series splits` method will be used: involves dividing the time series into blocks of equal length, and then using each block as a separate fold for cross-validation.

<img src="https://github.com/CorsiDanilo/big-data-computing-project/blob/main/notebooks/images/block-splits.png?raw=1">

In [13]:
# Get splitting parameters based on the choosen splitting method
splitting_info = train_validation_utilities.get_splitting_params(SPLITTING_METHOD)
splitting_info

{'split_type': 'block_splits', 'splits': 5}

## Default
The train / validation set will be splitted based on the splitting method chosen so that the model performance can be seen without any tuning by using different features (normalized and non)

In [14]:
# Get default parameters
params = train_validation_utilities.get_defaults_model_params(MODEL_NAME)
params

{'maxIter': [20], 'maxDepth': [5], 'stepSize': [0.1], 'seed': [42]}

### Without normalization

In [15]:
# Define model and features type
MODEL_TYPE = "default"
FEATURES_NORMALIZATION = False

In [16]:
# Choose base features
CHOSEN_FEATURES = BASE_FEATURES
CHOSEN_FEATURES_LABEL = BASE_FEATURES_LABEL

In [17]:
# Make predictions by using base features
default_train_results_base_features, default_valid_results_base_features, default_train_pred_base_features, default_valid_pred_base_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

100%|██████████| 1/1 [00:07<00:00,  7.23s/it]


In [18]:
default_train_results_base_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,train,block_splits,base_features,1,"(20981, 5246)","[20, 5, 0.1, 42]",40.389406,1631.304131,29.603821,0.003839,0.998943,0.998942,9.207020
1,GradientBoostingTreeRegressor,default,train,block_splits,base_features,2,"(20981, 5246)","[20, 5, 0.1, 42]",309.076131,95528.054699,201.629125,0.008074,0.999601,0.999601,6.362068
2,GradientBoostingTreeRegressor,default,train,block_splits,base_features,3,"(20981, 5246)","[20, 5, 0.1, 42]",278.665092,77654.233632,215.365986,0.004753,0.999223,0.999223,6.200807
3,GradientBoostingTreeRegressor,default,train,block_splits,base_features,4,"(20981, 5246)","[20, 5, 0.1, 42]",201.805783,40725.574171,144.032431,0.004747,0.999517,0.999517,6.081309
4,GradientBoostingTreeRegressor,default,train,block_splits,base_features,5,"(20981, 5246)","[20, 5, 0.1, 42]",101.896133,10382.821835,70.423817,0.003041,0.999514,0.999514,5.894922


In [19]:
default_valid_results_base_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,valid,block_splits,base_features,1,"(20981, 5246)","[20, 5, 0.1, 42]",703.982053,4.955907e+05,423.851196,0.037326,0.551738,0.551396,9.207020
1,GradientBoostingTreeRegressor,default,valid,block_splits,base_features,2,"(20981, 5246)","[20, 5, 0.1, 42]",1444.026316,2.085212e+06,943.145926,0.016255,0.808075,0.807929,6.362068
2,GradientBoostingTreeRegressor,default,valid,block_splits,base_features,3,"(20981, 5246)","[20, 5, 0.1, 42]",469.604442,2.205283e+05,367.936760,0.008685,0.987923,0.987914,6.200807
3,GradientBoostingTreeRegressor,default,valid,block_splits,base_features,4,"(20981, 5246)","[20, 5, 0.1, 42]",565.407766,3.196859e+05,265.517017,0.014720,0.631440,0.631159,6.081309
4,GradientBoostingTreeRegressor,default,valid,block_splits,base_features,5,"(20981, 5246)","[20, 5, 0.1, 42]",291.186323,8.478947e+04,231.624791,0.007695,0.837281,0.837157,5.894922


In [20]:
# Choose base and most additional correlated features
CHOSEN_FEATURES = BASE_AND_MOST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_MOST_CORR_FEATURES_LABEL

In [21]:
# Make predictions by using base and most additional correlated features
default_train_results_base_and_most_corr_features, default_valid_results_base_and_most_corr_features, default_train_pred_base_and_most_corr_features, default_valid_pred_base_and_most_corr_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

100%|██████████| 1/1 [00:07<00:00,  7.10s/it]


In [22]:
default_train_results_base_and_most_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,train,block_splits,base_and_most_corr_features,1,"(20981, 5246)","[20, 5, 0.1, 42]",38.978697,1519.338826,28.693899,0.003669,0.999015,0.999015,5.768668
1,GradientBoostingTreeRegressor,default,train,block_splits,base_and_most_corr_features,2,"(20981, 5246)","[20, 5, 0.1, 42]",297.901741,88745.447027,194.349218,0.007736,0.999629,0.999629,5.582110
2,GradientBoostingTreeRegressor,default,train,block_splits,base_and_most_corr_features,3,"(20981, 5246)","[20, 5, 0.1, 42]",269.186961,72461.620053,206.979172,0.004627,0.999275,0.999275,5.648754
3,GradientBoostingTreeRegressor,default,train,block_splits,base_and_most_corr_features,4,"(20981, 5246)","[20, 5, 0.1, 42]",180.880823,32717.872214,133.932149,0.004488,0.999612,0.999612,5.781864
4,GradientBoostingTreeRegressor,default,train,block_splits,base_and_most_corr_features,5,"(20981, 5246)","[20, 5, 0.1, 42]",92.092021,8480.940330,67.219848,0.002955,0.999603,0.999603,5.807318


In [23]:
default_valid_results_base_and_most_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,valid,block_splits,base_and_most_corr_features,1,"(20981, 5246)","[20, 5, 0.1, 42]",786.039424,6.178580e+05,475.722731,0.041890,0.441147,0.440721,5.768668
1,GradientBoostingTreeRegressor,default,valid,block_splits,base_and_most_corr_features,2,"(20981, 5246)","[20, 5, 0.1, 42]",1516.421682,2.299535e+06,1017.629034,0.017603,0.788349,0.788187,5.582110
2,GradientBoostingTreeRegressor,default,valid,block_splits,base_and_most_corr_features,3,"(20981, 5246)","[20, 5, 0.1, 42]",359.638480,1.293398e+05,299.707459,0.007111,0.992917,0.992911,5.648754
3,GradientBoostingTreeRegressor,default,valid,block_splits,base_and_most_corr_features,4,"(20981, 5246)","[20, 5, 0.1, 42]",561.666416,3.154692e+05,268.761212,0.014856,0.636301,0.636024,5.781864
4,GradientBoostingTreeRegressor,default,valid,block_splits,base_and_most_corr_features,5,"(20981, 5246)","[20, 5, 0.1, 42]",337.189055,1.136965e+05,267.561890,0.008894,0.781806,0.781640,5.807318


In [24]:
# Choose base and least additional correlated features
CHOSEN_FEATURES = BASE_AND_LEAST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_LEAST_CORR_FEATURES_LABEL

In [25]:
# Make predictions by using base and least additional correlated features
default_train_results_base_and_least_corr_features, default_valid_results_base_and_least_corr_features, default_train_pred_base_and_least_corr_features, default_valid_pred_base_and_least_corr_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

100%|██████████| 1/1 [00:07<00:00,  7.22s/it]


In [26]:
default_train_results_base_and_least_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,train,block_splits,base_and_least_corr_features,1,"(20981, 5246)","[20, 5, 0.1, 42]",39.198665,1536.535300,29.537930,0.003772,0.999004,0.999004,5.821974
1,GradientBoostingTreeRegressor,default,train,block_splits,base_and_least_corr_features,2,"(20981, 5246)","[20, 5, 0.1, 42]",300.722620,90434.094246,200.789196,0.008404,0.999622,0.999622,5.663334
2,GradientBoostingTreeRegressor,default,train,block_splits,base_and_least_corr_features,3,"(20981, 5246)","[20, 5, 0.1, 42]",263.087289,69214.921551,202.082575,0.004486,0.999307,0.999307,5.948411
3,GradientBoostingTreeRegressor,default,train,block_splits,base_and_least_corr_features,4,"(20981, 5246)","[20, 5, 0.1, 42]",177.453814,31489.856051,132.799309,0.004429,0.999627,0.999627,6.028208
4,GradientBoostingTreeRegressor,default,train,block_splits,base_and_least_corr_features,5,"(20981, 5246)","[20, 5, 0.1, 42]",93.211465,8688.377226,68.537501,0.002987,0.999593,0.999593,5.969177


In [27]:
default_valid_results_base_and_least_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,valid,block_splits,base_and_least_corr_features,1,"(20981, 5246)","[20, 5, 0.1, 42]",791.057750,6.257724e+05,479.534655,0.042251,0.433989,0.433557,5.821974
1,GradientBoostingTreeRegressor,default,valid,block_splits,base_and_least_corr_features,2,"(20981, 5246)","[20, 5, 0.1, 42]",1502.600755,2.257809e+06,1053.377238,0.018258,0.792189,0.792030,5.663334
2,GradientBoostingTreeRegressor,default,valid,block_splits,base_and_least_corr_features,3,"(20981, 5246)","[20, 5, 0.1, 42]",358.670127,1.286443e+05,298.334830,0.007049,0.992955,0.992950,5.948411
3,GradientBoostingTreeRegressor,default,valid,block_splits,base_and_least_corr_features,4,"(20981, 5246)","[20, 5, 0.1, 42]",574.660208,3.302344e+05,269.201279,0.014928,0.619279,0.618988,6.028208
4,GradientBoostingTreeRegressor,default,valid,block_splits,base_and_least_corr_features,5,"(20981, 5246)","[20, 5, 0.1, 42]",312.088184,9.739903e+04,257.763088,0.008575,0.813083,0.812940,5.969177


### With normalization

In [28]:
# Define model and features type
MODEL_TYPE = "default_norm"
FEATURES_NORMALIZATION = True

In [29]:
# Choose base features
CHOSEN_FEATURES = BASE_FEATURES
CHOSEN_FEATURES_LABEL = BASE_FEATURES_LABEL

In [30]:
# Make predictions by using base features
default_norm_train_results_base_features, default_norm_valid_results_base_features, default_norm_train_pred_base_features, default_norm_valid_pred_base_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

100%|██████████| 1/1 [00:07<00:00,  7.03s/it]


In [31]:
default_norm_train_results_base_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default_norm,train,block_splits,base_features_norm,1,"(20981, 5246)","[20, 5, 0.1, 42]",71.311294,5085.300722,51.117986,0.006480,0.996704,0.996703,6.173959
1,GradientBoostingTreeRegressor,default_norm,train,block_splits,base_features_norm,2,"(20981, 5246)","[20, 5, 0.1, 42]",451.566184,203912.018173,279.948563,0.011324,0.999149,0.999148,5.721828
2,GradientBoostingTreeRegressor,default_norm,train,block_splits,base_features_norm,3,"(20981, 5246)","[20, 5, 0.1, 42]",442.167212,195511.843051,321.696978,0.007143,0.998044,0.998043,5.466759
3,GradientBoostingTreeRegressor,default_norm,train,block_splits,base_features_norm,4,"(20981, 5246)","[20, 5, 0.1, 42]",274.414952,75303.565637,189.888327,0.006349,0.999107,0.999107,5.540562
4,GradientBoostingTreeRegressor,default_norm,train,block_splits,base_features_norm,5,"(20981, 5246)","[20, 5, 0.1, 42]",120.366046,14487.985145,88.735288,0.003829,0.999322,0.999321,5.790107


In [32]:
default_norm_valid_results_base_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default_norm,valid,block_splits,base_features_norm,1,"(20981, 5246)","[20, 5, 0.1, 42]",859.833016,7.393128e+05,546.615767,0.048715,0.331291,0.330781,6.173959
1,GradientBoostingTreeRegressor,default_norm,valid,block_splits,base_features_norm,2,"(20981, 5246)","[20, 5, 0.1, 42]",3026.178454,9.157756e+06,2344.325804,0.040819,0.157111,0.156468,5.721828
2,GradientBoostingTreeRegressor,default_norm,valid,block_splits,base_features_norm,3,"(20981, 5246)","[20, 5, 0.1, 42]",837.866991,7.020211e+05,657.271555,0.015659,0.961555,0.961525,5.466759
3,GradientBoostingTreeRegressor,default_norm,valid,block_splits,base_features_norm,4,"(20981, 5246)","[20, 5, 0.1, 42]",691.972976,4.788266e+05,356.842955,0.019593,0.447970,0.447548,5.540562
4,GradientBoostingTreeRegressor,default_norm,valid,block_splits,base_features_norm,5,"(20981, 5246)","[20, 5, 0.1, 42]",520.375064,2.707902e+05,426.325995,0.014145,0.480330,0.479933,5.790107


In [33]:
# Choose base and most additional correlated features
CHOSEN_FEATURES = BASE_AND_MOST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_MOST_CORR_FEATURES_LABEL

In [34]:
# Make predictions by using base and most additional correlated features
default_norm_train_results_base_and_most_corr_features, default_norm_valid_results_base_and_most_corr_features, default_norm_train_pred_base_and_most_corr_features, default_norm_valid_pred_base_and_most_corr_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

100%|██████████| 1/1 [00:07<00:00,  7.09s/it]


In [35]:
default_norm_train_results_base_and_most_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default_norm,train,block_splits,base_and_most_corr_features_norm,1,"(20981, 5246)","[20, 5, 0.1, 42]",64.104328,4109.364838,47.655016,0.006034,0.997336,0.997336,5.676656
1,GradientBoostingTreeRegressor,default_norm,train,block_splits,base_and_most_corr_features_norm,2,"(20981, 5246)","[20, 5, 0.1, 42]",366.720664,134484.045681,244.826553,0.010818,0.999439,0.999438,5.721667
2,GradientBoostingTreeRegressor,default_norm,train,block_splits,base_and_most_corr_features_norm,3,"(20981, 5246)","[20, 5, 0.1, 42]",406.356548,165125.644001,305.214172,0.006801,0.998348,0.998347,5.718045
3,GradientBoostingTreeRegressor,default_norm,train,block_splits,base_and_most_corr_features_norm,4,"(20981, 5246)","[20, 5, 0.1, 42]",254.296032,64666.472024,177.839920,0.005967,0.999233,0.999233,5.680793
4,GradientBoostingTreeRegressor,default_norm,train,block_splits,base_and_most_corr_features_norm,5,"(20981, 5246)","[20, 5, 0.1, 42]",108.688782,11813.251322,80.299756,0.003484,0.999447,0.999447,5.786098


In [36]:
default_norm_valid_results_base_and_most_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default_norm,valid,block_splits,base_and_most_corr_features_norm,1,"(20981, 5246)","[20, 5, 0.1, 42]",847.389740,7.180694e+05,541.732209,0.048283,0.350506,0.350010,5.676656
1,GradientBoostingTreeRegressor,default_norm,valid,block_splits,base_and_most_corr_features_norm,2,"(20981, 5246)","[20, 5, 0.1, 42]",3455.074352,1.193754e+07,2670.024616,0.046283,-0.098743,-0.099581,5.721667
2,GradientBoostingTreeRegressor,default_norm,valid,block_splits,base_and_most_corr_features_norm,3,"(20981, 5246)","[20, 5, 0.1, 42]",2020.761285,4.083476e+06,1443.603387,0.036180,0.776374,0.776203,5.718045
3,GradientBoostingTreeRegressor,default_norm,valid,block_splits,base_and_most_corr_features_norm,4,"(20981, 5246)","[20, 5, 0.1, 42]",3502.140970,1.226499e+07,1393.811397,0.069436,-13.140084,-13.150876,5.680793
4,GradientBoostingTreeRegressor,default_norm,valid,block_splits,base_and_most_corr_features_norm,5,"(20981, 5246)","[20, 5, 0.1, 42]",625.445615,3.911822e+05,532.217882,0.017669,0.249287,0.248714,5.786098


In [37]:
# Choose base and least additional correlated features
CHOSEN_FEATURES = BASE_AND_LEAST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_LEAST_CORR_FEATURES_LABEL

In [38]:
# Make predictions by using base and least additional correlated features
default_norm_train_results_base_and_least_corr_features, default_norm_valid_results_base_and_least_corr_features, default_norm_train_pred_base_and_least_corr_features, default_norm_valid_pred_base_and_least_corr_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

100%|██████████| 1/1 [00:07<00:00,  7.46s/it]


In [39]:
default_norm_train_results_base_and_least_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default_norm,train,block_splits,base_and_least_corr_features_norm,1,"(20981, 5246)","[20, 5, 0.1, 42]",75.368319,5680.383491,52.845630,0.006676,0.996318,0.996317,5.681024
1,GradientBoostingTreeRegressor,default_norm,train,block_splits,base_and_least_corr_features_norm,2,"(20981, 5246)","[20, 5, 0.1, 42]",349.454134,122118.192053,240.574912,0.010501,0.999490,0.999490,5.509035
2,GradientBoostingTreeRegressor,default_norm,train,block_splits,base_and_least_corr_features_norm,3,"(20981, 5246)","[20, 5, 0.1, 42]",569.351399,324161.015499,421.247249,0.009169,0.996756,0.996756,5.653017
3,GradientBoostingTreeRegressor,default_norm,train,block_splits,base_and_least_corr_features_norm,4,"(20981, 5246)","[20, 5, 0.1, 42]",246.825951,60923.049951,174.263338,0.005791,0.999278,0.999278,5.602180
4,GradientBoostingTreeRegressor,default_norm,train,block_splits,base_and_least_corr_features_norm,5,"(20981, 5246)","[20, 5, 0.1, 42]",129.749181,16834.850071,95.851931,0.004200,0.999212,0.999212,6.066210


In [40]:
default_norm_valid_results_base_and_least_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default_norm,valid,block_splits,base_and_least_corr_features_norm,1,"(20981, 5246)","[20, 5, 0.1, 42]",793.346638,6.293989e+05,614.523041,0.057522,0.430709,0.430274,5.681024
1,GradientBoostingTreeRegressor,default_norm,valid,block_splits,base_and_least_corr_features_norm,2,"(20981, 5246)","[20, 5, 0.1, 42]",3920.442123,1.536987e+07,3501.206379,0.062265,-0.414658,-0.415737,5.509035
2,GradientBoostingTreeRegressor,default_norm,valid,block_splits,base_and_least_corr_features_norm,3,"(20981, 5246)","[20, 5, 0.1, 42]",6529.872485,4.263923e+07,5624.559993,0.139351,-1.335081,-1.336863,5.653017
3,GradientBoostingTreeRegressor,default_norm,valid,block_splits,base_and_least_corr_features_norm,4,"(20981, 5246)","[20, 5, 0.1, 42]",1008.881386,1.017842e+06,734.258769,0.038960,-0.173451,-0.174347,5.602180
4,GradientBoostingTreeRegressor,default_norm,valid,block_splits,base_and_least_corr_features_norm,5,"(20981, 5246)","[20, 5, 0.1, 42]",2282.950029,5.211861e+06,2217.251217,0.073928,-9.002023,-9.009657,6.066210


In [41]:
# Define model information and evaluators to show
model_info = ['Model', 'Type', 'Dataset', 'Splitting', 'Features', 'Parameters']
evaluator_lst = ['RMSE', 'MSE', 'MAE', 'MAPE', 'R2', 'Adjusted_R2', 'Time']

In [42]:
# Define the results to show
default_comparison_lst = [default_valid_results_base_features, default_valid_results_base_and_most_corr_features, default_valid_results_base_and_least_corr_features, default_norm_valid_results_base_features, default_norm_valid_results_base_and_most_corr_features, default_norm_valid_results_base_and_least_corr_features]

# Show the comparison table
default_comparison_lst_df = pd.concat([train_validation_utilities.model_comparison(results, model_info, evaluator_lst) for results in default_comparison_lst])
default_comparison_lst_df

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,valid,block_splits,base_features,"[20, 5, 0.1, 42]",694.841380,6.411613e+05,446.415138,0.016936,0.763292,0.763111,6.749225
0,GradientBoostingTreeRegressor,default,valid,block_splits,base_and_most_corr_features,"[20, 5, 0.1, 42]",712.191012,6.951796e+05,465.876465,0.018071,0.728104,0.727897,5.717743
0,GradientBoostingTreeRegressor,default,valid,block_splits,base_and_least_corr_features,"[20, 5, 0.1, 42]",707.815405,6.879718e+05,471.642218,0.018212,0.730299,0.730093,5.886221
0,GradientBoostingTreeRegressor,default_norm,valid,block_splits,base_features_norm,"[20, 5, 0.1, 42]",1187.245300,2.269741e+06,866.276415,0.027786,0.475651,0.475251,5.738643
0,GradientBoostingTreeRegressor,default_norm,valid,block_splits,base_and_most_corr_features_norm,"[20, 5, 0.1, 42]",2090.162392,5.879052e+06,1316.277898,0.043570,-2.372532,-2.375106,5.716652
0,GradientBoostingTreeRegressor,default_norm,valid,block_splits,base_and_least_corr_features_norm,"[20, 5, 0.1, 42]",2907.098532,1.297364e+07,2538.359880,0.074405,-2.098901,-2.101266,5.702293


In [43]:
# Save the best default model results and predicitons
best_default_results = pd.concat([train_validation_utilities.model_comparison(results, model_info, evaluator_lst) for results in [default_valid_results_base_features]])
best_default_predictions = default_norm_valid_pred_base_features
best_default_results

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,valid,block_splits,base_features,"[20, 5, 0.1, 42]",694.84138,641161.296247,446.415138,0.016936,0.763292,0.763111,6.749225


In [44]:
# Define the choosen features
FEATURES_NORMALIZATION = False
CHOSEN_FEATURES = BASE_FEATURES
CHOSEN_FEATURES_LABEL = BASE_FEATURES_LABEL

## Tuned
Once the features and execution method are selected, the model will undergo hyperparameter tuning and cross validation to find the best configuration. 

In [45]:
# Get model grid parameters
params = train_validation_utilities.get_model_grid_params(MODEL_NAME)
params

{'maxIter': [3, 5, 10, 20, 30],
 'maxDepth': [3, 5, 10],
 'stepSize': [0.1, 0.3, 0.5, 0.7],
 'seed': [42]}

### Hyperparameter tuning

In [46]:
# Select the type of feature to be used
MODEL_TYPE = "hyp_tuning"

In [47]:
# Perform hyperparameter tuning
hyp_res = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)
hyp_res

100%|██████████| 60/60 [05:25<00:00,  5.43s/it]


Best parameters chosen for split [1/5]: [30, 3, 0.5, 42]


100%|██████████| 60/60 [05:23<00:00,  5.40s/it]


Best parameters chosen for split [2/5]: [5, 3, 0.5, 42]


100%|██████████| 60/60 [05:32<00:00,  5.55s/it]


...

100%|██████████| 60/60 [05:26<00:00,  5.45s/it]


Best parameters chosen for split [4/5]: [10, 10, 0.7, 42]


100%|██████████| 60/60 [05:32<00:00,  5.53s/it]

Best parameters chosen for split [5/5]: [3, 5, 0.1, 42]


,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,hyp_tuning,valid,block_splits,base_features,1,"(20981, 5246)","[30, 3, 0.5, 42]",669.223657,4.478603e+05,405.376688,0.035774,0.594910,0.594601,4.315647
1,GradientBoostingTreeRegressor,hyp_tuning,valid,block_splits,base_features,2,"(20981, 5246)","[5, 3, 0.5, 42]",1388.769437,1.928681e+06,957.891369,0.016773,0.822482,0.822347,0.915803
2,GradientBoostingTreeRegressor,hyp_tuning,valid,block_splits,base_features,3,"(20981, 5246)","[3, 10, 0.7, 42]",453.469894,2.056349e+05,336.834731,0.007993,0.988739,0.988730,1.651005
3,GradientBoostingTreeRegressor,hyp_tuning,valid,block_splits,base_features,4,"(20981, 5246)","[10, 10, 0.7, 42]",527.134761,2.778711e+05,280.035001,0.015314,0.679648,0.679403,5.357254
4,GradientBoostingTreeRegressor,hyp_tuning,valid,block_splits,base_features,5,"(20981, 5246)","[3, 5, 0.1, 42]",268.082011,7.186796e+04,217.636373,0.007259,0.862079,0.861974,0.785339


---
Since during this stage will be used the Block split / Walk forward split method of the dataset I compute a score for each parameter chosen by each split, assigning weights based on:
   * Their `frequency` for each split (if the same parameters are chosen from several splits, these will have greater weight) 
   * The `split` they belong to (the closer the split is to today's date the more weight they will have)
   * Their `RMSE value` for each split (the lower this is, the more weight they will have)
   
   Then, the overall score will be calculated by putting together these three weights for each parameter and the one with the best score will be the chosen parameter.

In [48]:
# Show parameters score
grouped_scores, best_params = train_validation_utilities.choose_best_params(hyp_res)
grouped_scores

,Split weight,RMSE weight,Frequency weight,Final score
Parameters,,,,
"(3, 5, 0.1, 42)",1.0,0.806964,1.0,0.806964
"(10, 10, 0.7, 42)",0.8,0.620430,1.0,0.496344
"(3, 10, 0.7, 42)",0.6,0.673474,1.0,0.404084
"(30, 3, 0.5, 42)",0.2,0.518118,1.0,0.103624
"(5, 3, 0.5, 42)",0.4,0.000000,1.0,0.000000


In [49]:
# Print best parameters
print(f"Best parameters: {best_params}")

Best parameters: (3, 5, 0.1, 42)


### Cross validation

In [50]:
MODEL_TYPE = "cross_val"

In [51]:
# Get tuned parameters
params = train_validation_utilities.get_best_model_params(best_params, MODEL_NAME)
params

{'maxIter': [3], 'maxDepth': [5], 'stepSize': [0.1], 'seed': [42]}

In [52]:
# Perform cross validation
cv_train_result, cv_valid_result, cv_train_pred, cv_valid_pred = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

100%|██████████| 1/1 [00:01<00:00,  1.87s/it]


In [53]:
cv_train_result

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,cross_val,train,block_splits,base_features,1,"(20981, 5246)","[3, 5, 0.1, 42]",56.743278,3219.799556,43.478016,0.005688,0.997913,0.997913,1.029752
1,GradientBoostingTreeRegressor,cross_val,train,block_splits,base_features,2,"(20981, 5246)","[3, 5, 0.1, 42]",447.480635,200238.918509,304.889282,0.012532,0.999164,0.999164,0.969974
2,GradientBoostingTreeRegressor,cross_val,train,block_splits,base_features,3,"(20981, 5246)","[3, 5, 0.1, 42]",381.963390,145896.031637,304.208519,0.006748,0.998540,0.998540,0.982596
3,GradientBoostingTreeRegressor,cross_val,train,block_splits,base_features,4,"(20981, 5246)","[3, 5, 0.1, 42]",278.919447,77796.057642,206.231259,0.006811,0.999078,0.999077,0.954204
4,GradientBoostingTreeRegressor,cross_val,train,block_splits,base_features,5,"(20981, 5246)","[3, 5, 0.1, 42]",145.766288,21247.810836,109.234568,0.004704,0.999005,0.999005,0.929116


In [54]:
cv_valid_result

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,cross_val,valid,block_splits,base_features,1,"(20981, 5246)","[3, 5, 0.1, 42]",721.751105,5.209247e+05,435.410132,0.038350,0.528823,0.528464,1.029752
1,GradientBoostingTreeRegressor,cross_val,valid,block_splits,base_features,2,"(20981, 5246)","[3, 5, 0.1, 42]",1552.556298,2.410431e+06,1051.573735,0.018324,0.778142,0.777972,0.969974
2,GradientBoostingTreeRegressor,cross_val,valid,block_splits,base_features,3,"(20981, 5246)","[3, 5, 0.1, 42]",473.943429,2.246224e+05,373.322824,0.008794,0.987699,0.987689,0.982596
3,GradientBoostingTreeRegressor,cross_val,valid,block_splits,base_features,4,"(20981, 5246)","[3, 5, 0.1, 42]",645.077930,4.161255e+05,313.342293,0.017320,0.520256,0.519890,0.954204
4,GradientBoostingTreeRegressor,cross_val,valid,block_splits,base_features,5,"(20981, 5246)","[3, 5, 0.1, 42]",268.082011,7.186796e+04,217.636373,0.007259,0.862079,0.861974,0.929116


In [55]:
# Define the results to show
tuned_comparison_lst = [cv_valid_result]

# Show the comparison table
tuned_comparison_lst_df = pd.concat([train_validation_utilities.model_comparison(results, model_info, evaluator_lst) for results in tuned_comparison_lst])
tuned_comparison_lst_df

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,cross_val,valid,block_splits,base_features,"[3, 5, 0.1, 42]",732.282155,728794.318477,478.257071,0.018009,0.7354,0.735198,0.973128


# Comparison table
Visualization of model performance at various stages of train / validation

In [56]:
# Concatenate final results into Pandas dataset
final_comparison_lst_df = pd.DataFrame(pd.concat([default_comparison_lst_df, tuned_comparison_lst_df], ignore_index=True))
final_comparison_lst_df

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,valid,block_splits,base_features,"[20, 5, 0.1, 42]",694.841380,6.411613e+05,446.415138,0.016936,0.763292,0.763111,6.749225
1,GradientBoostingTreeRegressor,default,valid,block_splits,base_and_most_corr_features,"[20, 5, 0.1, 42]",712.191012,6.951796e+05,465.876465,0.018071,0.728104,0.727897,5.717743
2,GradientBoostingTreeRegressor,default,valid,block_splits,base_and_least_corr_features,"[20, 5, 0.1, 42]",707.815405,6.879718e+05,471.642218,0.018212,0.730299,0.730093,5.886221
3,GradientBoostingTreeRegressor,default_norm,valid,block_splits,base_features_norm,"[20, 5, 0.1, 42]",1187.245300,2.269741e+06,866.276415,0.027786,0.475651,0.475251,5.738643
4,GradientBoostingTreeRegressor,default_norm,valid,block_splits,base_and_most_corr_features_norm,"[20, 5, 0.1, 42]",2090.162392,5.879052e+06,1316.277898,0.043570,-2.372532,-2.375106,5.716652
5,GradientBoostingTreeRegressor,default_norm,valid,block_splits,base_and_least_corr_features_norm,"[20, 5, 0.1, 42]",2907.098532,1.297364e+07,2538.359880,0.074405,-2.098901,-2.101266,5.702293
6,GradientBoostingTreeRegressor,cross_val,valid,block_splits,base_features,"[3, 5, 0.1, 42]",732.282155,7.287943e+05,478.257071,0.018009,0.735400,0.735198,0.973128


# Model accuracy

Since predicting the price accurately is very difficult I also saw how good the models are at predicting whether the price will go up or down in this way:

For each prediction let's consider the actual market-price, next-market-price and our predicted next-market-price (prediction).
I compute whether the current prediction is correct (1) or not (0):

$$ 
prediction\_is\_correct
= 
\begin{cases}
0 \text{ if [(market-price > next-market-price) and (market-price < prediction)] or [(market-price < next-market-price) and (market-price > prediction)]} \\
1 \text{ if [(market-price > next-market-price) and (market-price > prediction)] or [(market-price < next-market-price) and (market-price < prediction)]}
\end{cases}
$$

After that I count the number of correct prediction:
$$ 
correct\_predictions
= 
\sum_{i=0}^{total\_rows} prediction\_is\_correct
$$

Finally I compute the percentage of accuracy of the model:
$$
\\ 
accuracy 
= 
(correct\_predictions / total\_rows) 
* 100
$$

In [57]:
# Convert the pandas dataset to a PySpark dataset
best_default_pred_spark = spark.createDataFrame(best_default_predictions)
validated_pred_spark = spark.createDataFrame(cv_valid_pred)

# Compute model accuracy
default_accuracy = train_validation_utilities.model_accuracy(best_default_pred_spark)
validated_accuracy = train_validation_utilities.model_accuracy(validated_pred_spark)

# Shows whether features are normalised or not
if FEATURES_NORMALIZATION:
    NEW_CHOSEN_FEATURES_LABEL = CHOSEN_FEATURES_LABEL + "_norm"
    CHOSEN_FEATURES_LABEL = NEW_CHOSEN_FEATURES_LABEL
    
# Saving accuracy data into dataframe
accuracy_data = {
    'Model': MODEL_NAME,
    'Features': CHOSEN_FEATURES_LABEL,
    'Splitting': SPLITTING_METHOD,
    'Accuracy (default)': default_accuracy,
    'Accuracy (tuned)': validated_accuracy
}
accuracy_data_df = pd.DataFrame(accuracy_data, index=['Model'])

print(f"Percentage of correct predictions for {MODEL_NAME} with {CHOSEN_FEATURES_LABEL} and {SPLITTING_METHOD} (default): {default_accuracy:.2f}%")
print(f"Percentage of correct predictions for {MODEL_NAME} with {CHOSEN_FEATURES_LABEL} and {SPLITTING_METHOD} (tuned): {validated_accuracy:.2f}%")

Percentage of correct predictions for GradientBoostingTreeRegressor with base_features and block_splits (default): 50.41%
Percentage of correct predictions for GradientBoostingTreeRegressor with base_features and block_splits (tuned): 50.17%


In [58]:
# Concatenate default and tuned results
default_tuned_results = [best_default_results, cv_valid_result]
default_tuned_results_df = pd.concat([train_validation_utilities.model_comparison(results, model_info, evaluator_lst) for results in default_tuned_results])
default_tuned_results_df

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,valid,block_splits,base_features,"[20, 5, 0.1, 42]",694.841380,641161.296247,446.415138,0.016936,0.763292,0.763111,6.749225
0,GradientBoostingTreeRegressor,cross_val,valid,block_splits,base_features,"[3, 5, 0.1, 42]",732.282155,728794.318477,478.257071,0.018009,0.735400,0.735198,0.973128


# Saving final results

In [59]:
# Save all final comparison results
final_comparison_lst_df.to_csv(ALL_MODEL_RESULTS, index=False)

In [60]:
# Save relevant results (default and tuned results)
default_tuned_results_df.to_csv(REL_MODEL_RESULTS, index=False)

In [61]:
# Saving accuracy results
accuracy_data_df.to_csv(MODEL_ACCURACY_RESULTS, index=False)

In [62]:
# Export notebook in html format (remember to save the notebook and change the model name)
if LOCAL_RUNNING:
    !jupyter nbconvert --to html 3-block-split_{MODEL_NAME}.ipynb --output 3-block-split_{MODEL_NAME} --output-dir='./exports'

d:\Documents\Repository\BDC\project\.bdc\Lib\site-packages\traitlets\traitlets.py:2548: FutureWarning: Supporting extra quotes around strings is deprecated in traitlets 5.0. You can use './exports' instead of "'./exports'" if you require traitlets >=5.
  warn(
[NbConvertApp] Converting notebook 3-block-split_GradientBoostingTreeRegressor.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 1 image(s).
[NbConvertApp] Writing 436334 bytes to exports\3-block-split_GradientBoostingTreeRegressor.html
